In [1]:
from DatasetHandler.BiwiBrowser import *
from LSTM_VGG16.LSTM_VGG16Helper import *
%matplotlib inline

/home/mcicek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
output_begin = 4
num_outputs = 1

In [22]:
timesteps = 16 # TimeseriesGenerator Handles overlapping
in_epochs = 30
out_epochs = 1
train_batch_size = 10
test_batch_size = 10

In [23]:
subjectList = [9] #, 2, 3, 4, 5, 7, 8, 11, 12, 14 except [6, 13, 10, ]
testSubjects = [1]

In [5]:
num_datasets = len(subjectList)

In [33]:
def getFinalModel(num_outputs = num_outputs):
    inp = (224, 224, 3) # BIWI_Frame_Shape
    vgg_model = VGG16(weights='imagenet', input_shape = inp, include_top=False) #BIWI_Frame_Shape
    """"
    vgg_model.layers.pop()
    vgg_model.outputs = [vgg_model.layers[-1].output]#
    vgg_model.output_layers = [vgg_model.layers[-1]]#
    vgg_model.layers[-1].outbound_nodes = []#
    nb_pretrained_layers = len(vgg_model.layers)
    """
    for layer in vgg_model.layers: #
        layer.trainable = False#
    #print(nb_pretrained_layers)
    #vgg_model.summary()
    rnn = Sequential()
    rnn.add(TimeDistributed(vgg_model, input_shape=(timesteps, inp[0], inp[1], inp[2]), name = 'tdVGG16')) 
    rnn.add(TimeDistributed(Flatten()))
    
   # rnn.add(TimeDistributed(Dense(4096, activation='relu'), name = 'fc1024')), activation='relu'
   # rnn.add(TimeDistributed(Dense(4096, activation='relu'), name = 'fc104'))    
   # rnn.add(TimeDistributed(Dropout(0.25)))
   # rnn.add(TimeDistributed(Dense(124, activation='relu'), name = 'fc10'))#
   # rnn.add(TimeDistributed(Dropout(0.25)))

    rnn.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25))
   # rnn.add(Flatten())
    rnn.add(Dense(num_outputs))
    #print(len(rnn.layers))
    for layer in rnn.layers[:-2]:#
        layer.trainable = False#
    rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return rnn

In [34]:
keras.backend.clear_session()#
full_model = getFinalModel(num_outputs = num_outputs)

In [35]:
full_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tdVGG16 (TimeDistributed)    (None, 16, 7, 7, 512)     14714688  
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 25088)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               12911104  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 27,625,921
Trainable params: 12,911,233
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
full_model = trainImageModelForEpochs(full_model, out_epochs, subjectList, testSubjects, timesteps, False, output_begin, num_outputs, batch_size = train_batch_size, in_epochs = in_epochs)

All frames and annotations from 1 datasets have been read by 2019-01-10 01:26:23.623494
1. set (Dataset 9) being trained for epoch 1!
Epoch 1/30
87/87 [==============================] - 59s 678ms/step - loss: 0.1640 - mean_absolute_error: 0.3368
Epoch 2/30
87/87 [==============================] - 61s 700ms/step - loss: 0.1171 - mean_absolute_error: 0.2902
Epoch 3/30
87/87 [==============================] - 61s 705ms/step - loss: 0.1213 - mean_absolute_error: 0.2989
Epoch 4/30
87/87 [==============================] - 61s 704ms/step - loss: 0.1209 - mean_absolute_error: 0.2992
Epoch 5/30
87/87 [==============================] - 61s 706ms/step - loss: 0.1212 - mean_absolute_error: 0.2989
Epoch 6/30
75/87 [========================>.....] - ETA: 8s - loss: 0.1181 - mean_absolute_error: 0.2973

In [ ]:
test_generators, test_labelSets = getTestBiwiForImageModel(testSubjects, timesteps, False, output_begin, num_outputs, batch_size = test_batch_size)
test_gen, test_labels = test_generators[0], test_labelSets[0] #[1]

In [ ]:
predictions = full_model.predict_generator(test_gen, verbose = 1)
#predictions = full_model.predict(test_gen[0][0], verbose = 1)
output1 = numpy.concatenate((test_labels[timesteps:, :1], predictions[:, :1]), axis=1)

In [ ]:
print([i[0] for i in predictions[:10]])

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(output1)

In [ ]:
len(test_gen[0][0])

In [ ]:
test_gen[23][0][0][0][124][110]